* Data from https://opendata.dwd.de/weather/satellite/radiation/sid/

In [1]:
using YAXArrays,NetCDF

In [44]:
open_dataset("SIDin202311181200FDv3.nc")

YAXArray Dataset
Shared Axes: 
Ti Sampled{DateTime} DateTime[DateTime("2023-11-18T12:00:00")] ForwardOrdered Irregular Points
Variables: 

SID
 with dimensions: 
  Dim{:lon} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points
DNI
 with dimensions: 
  Dim{:lon} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Pointsatime, 
Properties: Dict{String, Any}("Version" => "SPECMAGIC_now Version 3.0", "Conventions" => "CF-1.6")


In [2]:
data_SID_late=open_dataset("SIDin202311181630FDv3.nc")["SID"]
data_SID_early=open_dataset("SIDin202311181200FDv3.nc")["SID"]

2601×2601×1 YAXArray{Int16,3} with dimensions: 
  Dim{:lon} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[DateTime("2023-11-18T12:00:00")] ForwardOrdered Irregular Points
units: Watt m-2
name: SID
Total size: 12.9 MB


In [3]:
using DimensionalData
lons=lookup(data_SID_late,:lon)
lats=lookup(data_SID_late,:lat)

Sampled{Float32} ForwardOrdered Regular Points
wrapping: -65.0f0:0.05f0:65.0f0

In [4]:
using GeoJSON
worldCountries = GeoJSON.read("../ne_110m_admin_0_countries.geojson")


FeatureCollection with 177 Features

In [5]:
using GeoMakie,GLMakie

In [6]:
data_DNI_early=open_dataset("SIDin202311181200FDv3.nc")["DNI"]
data_SID_early=open_dataset("SIDin202311181200FDv3.nc")["SID"]


2601×2601×1 YAXArray{Int16,3} with dimensions: 
  Dim{:lon} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[DateTime("2023-11-18T12:00:00")] ForwardOrdered Irregular Points
units: Watt m-2
name: SID
Total size: 12.9 MB


In [51]:
data_DNI_early

2601×2601×1 YAXArray{Int16,3} with dimensions: 
  Dim{:lon} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float32} -65.0f0:0.05f0:65.0f0 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[DateTime("2023-11-18T12:00:00")] ForwardOrdered Irregular Points
units: Watt m-2
name: DNI
Total size: 12.9 MB


In [10]:
fig=Figure()
ax=GeoAxis(fig[1,1],lonlims=(-65,65),latlims=(-65,65))
s=surface!(ax,lons,lats,data_DNI_early.data[:,:,1],colormap=cgrad(:fire,rev=false),colorrange=(0,900),shading=false)
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :gray75,
    strokewidth = 0.5)
translate!(hm1, 0, 0, 1000)

Colorbar(fig[1,2],s,label="Direct normalised radiation in Watt/m²")
fig

In [11]:
save("dni.png",fig)

In [12]:
using GLMakie
using Makie.GeometryBasics
using Makie.GeometryBasics: Tesselation, uv_normal_mesh
m = Makie.GeometryBasics.uv_normal_mesh(
    Makie.GeometryBasics.Tesselation(
        Makie.GeometryBasics.Sphere(
            Point3f(0), 1f0
        ),
        200
    )
);
p = decompose(Point3f, m)
uv = decompose_uv(m)
norms = decompose_normals(m);

In [83]:
m

Mesh{3, Float32, Triangle}:
 Triangle(Float32[0.0, 0.0, 1.0], Float32[0.015786242, 0.0, 0.99987537], Float32[0.015778374, 0.0004983488, 0.99987537])
 Triangle(Float32[0.0, 0.0, 1.0], Float32[0.015778374, 0.0004983488, 0.99987537], Float32[0.0, 0.0, 1.0])
 Triangle(Float32[0.015786242, 0.0, 0.99987537], Float32[0.03156855, 0.0, 0.9995016], Float32[0.031552814, 0.0009965734, 0.9995016])
 Triangle(Float32[0.015786242, 0.0, 0.99987537], Float32[0.031552814, 0.0009965734, 0.9995016], Float32[0.015778374, 0.0004983488, 0.99987537])
 Triangle(Float32[0.03156855, 0.0, 0.9995016], Float32[0.04734299, 0.0, 0.9988787], Float32[0.047319394, 0.0014945496, 0.9988787])
 Triangle(Float32[0.03156855, 0.0, 0.9995016], Float32[0.047319394, 0.0014945496, 0.9988787], Float32[0.031552814, 0.0009965734, 0.9995016])
 Triangle(Float32[0.04734299, 0.0, 0.9988787], Float32[0.06310563, 0.0, 0.9980069], Float32[0.06307418, 0.0019921532, 0.9980069])
 Triangle(Float32[0.04734299, 0.0, 0.9988787], Float32[0.06307418,

In [13]:
lon_new=size(-180:0.05:180,1)
lat_new=size(-90:0.05:90.0,1)

3601

In [99]:
size(65:0.05:90.0)

(501,)

In [15]:
testing=zeros(lon_new,lat_new)

7201×3601 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱       ⋮                        ⋮
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [16]:
testing[2300:4900,500:3100].=data_DNI_early.data[:,:,1]

2601×2601 view(::Matrix{Float64}, 2300:4900, 500:3100) with eltype Float64:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱       ⋮                        ⋮
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0 

In [76]:
function latLon2Position(position, radius = 1) 
    function inner_latlon(loc)
        lng,lat=loc
        @show lat, lng
        #phi = (90 - lat) * π / 180
        #theta = (180 - lng) * π/ 180
        phi=(180+lng)*π/180
        theta=(90+lat)*π/180
        x=radius*sin(phi)*cos(theta)
        y=radius*cos(phi)*cos(theta)
        z= radius*sin(theta)
        #x = radius * sin(phi) * cos(theta)
        #y = radius * cos(phi)
        #z = radius * sin(phi) * sin(theta)
        return Point3f(x,y,z)
    end
    if typeof(position)<:Vector
        result=[]
        for l in position
            push!(result,inner_latlon(l))
        end
        return result
    end
    return inner_latlon(position)
            
end

latLon2Position (generic function with 4 methods)

In [134]:
fig = Figure(resolution = (800,800), backgroundcolor=:snow2)
ax, plt_obj = mesh(fig[1,1], uv_normal_mesh(Tesselation(Sphere(Point3f(0), 0.99),128));
    color = (:white,0.1), transparency=true,
    axis = (type = LScene, show_axis = false)
    )
    temperature_plot = mesh!(
    m;
    color = rotl90(testing),
    colorrange = (0,900),
    colormap = :reds, #cmap,
    shading = true,
    transparency=false,
)
mesh!(fig[1,1],gb_mesh,  color = color)
fig

In [107]:
fig=Figure()
ax=Axis3(fig[1,1])
for i in w_new
    for x in i
    hm1=mesh!(
        ax, x[1],x[2],x[3];
        color= :black,
        strokecolor = :black,
        strokewidth = 0.25)
    end
    end
fig

In [102]:
w_new

148-element Vector{Any}:
 Vector{Any}[[Float32[-0.009248186, -0.013760819, 0.99986255], Float32[-0.010362399, -0.015320963, 0.99982893], Float32[-0.033037595, -0.042747676, 0.9985395], Float32[-0.03927871, -0.050698217, 0.9979413], Float32[-0.051534545, -0.06318258, 0.99667054], Float32[-0.064424194, -0.08029805, 0.9946868], Float32[-0.07066879, -0.08789497, 0.99361986], Float32[-0.07565878, -0.0919775, 0.9928826], Float32[-0.07857018, -0.095414996, 0.992332], Float32[-0.08471618, -0.103891954, 0.99097407]  …  Float32[-0.027059209, -0.04549506, 0.99859804], Float32[-0.024880959, -0.04219285, 0.9987996], Float32[-0.021356845, -0.03630066, 0.99911267], Float32[-0.021356724, -0.03630045, 0.99911267], Float32[-0.0204104, -0.03429547, 0.9992033], Float32[-0.015187769, -0.025461428, 0.9995604], Float32[-0.010026268, -0.017076315, 0.9998039], Float32[-0.009058304, -0.01521363, 0.99984324], Float32[-0.009465777, -0.015227432, 0.99983925], Float32[-0.009248186, -0.013760819, 0.99986255]]]
 Vect

In [113]:
GeometryBasics.Mesh(convert(Vector{Vector{Point3f}},w_new[42]))

MethodError: MethodError: no method matching GeometryBasics.Mesh(::Vector{Vector{Point{3, Float32}}})

Closest candidates are:
  GeometryBasics.Mesh(!Matched::AbstractVector{<:AbstractPoint}, !Matched::AbstractVector{<:Integer})
   @ GeometryBasics ~/.julia/packages/GeometryBasics/b8075/src/basic_types.jl:417
  GeometryBasics.Mesh(!Matched::AbstractVector{<:AbstractPoint}, !Matched::AbstractVector{<:Integer}, !Matched::Any)
   @ GeometryBasics ~/.julia/packages/GeometryBasics/b8075/src/basic_types.jl:417
  GeometryBasics.Mesh(!Matched::AbstractVector{<:AbstractPoint}, !Matched::AbstractVector{<:Integer}, !Matched::Any, !Matched::Any)
   @ GeometryBasics ~/.julia/packages/GeometryBasics/b8075/src/basic_types.jl:417
  ...


In [17]:
using LinearAlgebra, GLMakie, FileIO
# Create vertices for a Sphere
r = 1
n = 30
θ = LinRange(0, pi, n)
φ2 = LinRange(0, 2pi, 2 * n)
x2 = [r * cos(φv) * sin(θv) for θv in θ, φv in φ2]
y2 = [r * sin(φv) * sin(θv) for θv in θ, φv in φ2]
z2 = [r * cos(θv) for θv in θ, φv in 2φ2]
points = vec([Point3f(xv, yv, zv) for (xv, yv, zv) in zip(x2, y2, z2)])

# The coordinates form a matrix, so to connect neighboring vertices with a face
# we can just use the faces of a rectangle with the same dimension as the matrix:
faces = decompose(QuadFace{GLIndex}, Tesselation(Rect(0, 0, 1, 1), size(z2)))
# Normals of a centered sphere are easy, they're just the vertices normalized.
normals = normalize.(points)

# Now we generate UV coordinates, which map the image (texture) to the vertices.
# (0, 0) means lower left edge of the image, while (1, 1) means upper right corner.
function gen_uv(shift)
    return vec(map(CartesianIndices(size(z2))) do ci
        tup = ((ci[1], ci[2]) .- 1) ./ ((size(z2) .* shift) .- 1)
        return Vec2f(reverse(tup))
    end)
end

# We add some shift to demonstrate how UVs work:
uv = gen_uv(0.0)
# We can use a Buffer to update single elements in an array directly on the GPU
# with GLMakie. They work just like normal arrays, but forward any updates written to them directly to the GPU
uv_buff = Buffer(uv)
gb_mesh = GeometryBasics.Mesh(meta(points; uv=uv_buff, normals), faces)

Mesh{3, Float32, Quad}:
 Quad(Float32[0.0, 0.0, 1.0], Float32[0.10811902, 0.0, 0.99413794], Float32[0.107506506, 0.011492347, 0.99413794], Float32[0.0, 0.0, 1.0])
 Quad(Float32[0.10811902, 0.0, 0.99413794], Float32[0.21497044, 0.0, 0.97662055], Float32[0.21375258, 0.022849957, 0.97662055], Float32[0.107506506, 0.011492347, 0.99413794])
 Quad(Float32[0.21497044, 0.0, 0.97662055], Float32[0.31930152, 0.0, 0.9476532], Float32[0.31749263, 0.03393967, 0.9476532], Float32[0.21375258, 0.022849957, 0.97662055])
 Quad(Float32[0.31930152, 0.0, 0.9476532], Float32[0.4198891, 0.0, 0.9075754], Float32[0.41751036, 0.044631477, 0.9075754], Float32[0.31749263, 0.03393967, 0.9476532])
 Quad(Float32[0.4198891, 0.0, 0.9075754], Float32[0.51555383, 0.0, 0.8568572], Float32[0.51263314, 0.054800015, 0.8568572], Float32[0.41751036, 0.044631477, 0.9075754])
 Quad(Float32[0.51555383, 0.0, 0.8568572], Float32[0.60517424, 0.0, 0.79609305], Float32[0.6017458, 0.06432608, 0.79609305], Float32[0.51263314, 0.0548000

In [43]:

data = load(Makie.assetpath("earth.png"))
color = Sampler(rotl90(data'), x_repeat=:mirrored_repeat,y_repeat=:repeat)
f=Figure()
Label(f[1,1],text="Direct normalised radiation\n18.11.2023 12 UTC",fontsize=30)
ax=LScene(f[2,1],show_axis=false,width=800,height=800)
mesh!(ax,gb_mesh,  color = color,shading=false)
uv_buff[1:end] = gen_uv(1)
Makie.resize_to_layout!(f)
f

In [44]:
dni = mesh!(
    ax,m,
    color = rotl90(testing),
    colorrange = (0,900),
    colormap = cgrad(:bilbao,alpha=0.5,rev=true), #cmap,
    shading = false,
    transparency=false,
)
Colorbar(f[2,2],dni,label="Direct normalised radiation in Watt/m²")
Label(f[3,1],text="Data: © Deutscher Wetterdienst\nCode snippets by Makie and Lazaro Alonso",justification=:left)


Label()

In [45]:
Makie.resize_to_layout!(f)


(931.328f0, 987.18005f0)

In [24]:
f

In [46]:
save("atmosphere.png",f)

In [77]:
w_new =[]
for w in worldCountries
    try
     push!(w_new,latLon2Position.([(w.geometry.coordinates...)]))
        
    catch
        @show w.geometry.coordinates
    end
end

In [70]:
w_new

148-element Vector{Any}:
 Vector{Any}[[Float32[-0.8298621, 0.5577988, 1.0], Float32[-0.8281865, 0.5602432, 1.0], Float32[-0.7900819, 0.6115089, 1.0], Float32[-0.7888816, 0.6124505, 1.0], Float32[-0.77233994, 0.63205934, 1.0], Float32[-0.7758435, 0.6257947, 1.0], Float32[-0.77436817, 0.6266007, 1.0], Float32[-0.7667936, 0.6352698, 1.0], Float32[-0.76603806, 0.6356741, 1.0], Float32[-0.76800793, 0.6319575, 1.0]  …  Float32[-0.85826385, 0.5111882, 1.0], Float32[-0.8603501, 0.50795424, 1.0], Float32[-0.8611329, 0.50708216, 1.0], Float32[-0.8611329, 0.50708216, 1.0], Float32[-0.85864764, 0.5114177, 1.0], Float32[-0.85843813, 0.51228476, 1.0], Float32[-0.86217576, 0.5063214, 1.0], Float32[-0.85909444, 0.5115909, 1.0], Float32[-0.849147, 0.527937, 1.0], Float32[-0.8298621, 0.5577988, 1.0]]]
 Vector{Any}[[Float32[-0.8756357, -0.15066719, 1.0], Float32[-0.8761725, -0.15065917, 1.0], Float32[-0.87767035, -0.15099186, 1.0], Float32[-0.8893802, -0.15104158, 1.0], Float32[-0.8797511, -0.20738979, 1